### BigQuery Table Definitions for Stock Market Project


In [1]:
#assign dependencies and constants
from google.cloud import bigquery


In [5]:
def create_option_bq_table(stock_symbol):
    """
    Create a BigQuery table in the stock_options dataset for the supplied symbol
    """
    client = bigquery.Client()
    dataset_ref = client.dataset('option_quotes')

    schema = [
        bigquery.SchemaField("quote_date", "DATE", mode="REQUIRED", description="market closing date for which the data was captured"),
        bigquery.SchemaField("expiry_date", "DATE", mode="REQUIRED", description="date on which the options expiry"),
        bigquery.SchemaField("days_to_expiry", "INT64", mode="REQUIRED", description="calendar days between quote_date and expiry_date not including the quote_date"),
        bigquery.SchemaField("underlying_price", "NUMERIC", mode="REQUIRED", description="the closing price of the underlying asset"),
        bigquery.SchemaField("atm_price", "NUMERIC", mode="REQUIRED", description="adjusted underlying price such that the implied volatility of calls an puts are the same"),
        bigquery.SchemaField("atm_iv", "NUMERIC", mode="REQUIRED", description="the implied volatility for both calls and puts using the atm_price as the current price"),
        bigquery.SchemaField(
            "strike_prices",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("strike_price", "NUMERIC", mode="REQUIRED", description="price at which an option can be excercised"),
                bigquery.SchemaField("call_bid", "NUMERIC", mode="REQUIRED", description="closing bid price for the call option"),
                bigquery.SchemaField("call_ask", "NUMERIC", mode="REQUIRED", description="closing ask price for the call option"),
                bigquery.SchemaField("call_volume", "NUMERIC", mode="REQUIRED", description="number of call contracts traded"),
                bigquery.SchemaField("call_open_interest", "NUMERIC", mode="REQUIRED", description="number of open call contracts"),
                bigquery.SchemaField("call_moneyness", "NUMERIC", mode="REQUIRED", description="probability of call option closing in the money on expiry date (based on atm_iv)"),
                bigquery.SchemaField("call_iv", "NUMERIC", mode="REQUIRED", description="implied volatiity of the call option using the midpoint between the call bid and call ask price"),
                bigquery.SchemaField("put_bid", "NUMERIC", mode="REQUIRED", description="closing bid price for the put option"),
                bigquery.SchemaField("put_ask", "NUMERIC", mode="REQUIRED", description="closing ask price for the put option"),
                bigquery.SchemaField("put_volume", "NUMERIC", mode="REQUIRED", description="number of put contracts traded"),
                bigquery.SchemaField("put_open_interest", "NUMERIC", mode="REQUIRED", description="number of open call contracts"),
                bigquery.SchemaField("put_moneyness", "NUMERIC", mode="REQUIRED", description="probability of put option closing in the money on expiry date (based on atm_iv)"),
                bigquery.SchemaField("put_iv", "NUMERIC", mode="REQUIRED", description="implied volatiity of the put option using the midpoint between the put bid and put ask price"),
            ],
        ),
        bigquery.SchemaField("sampling_key", "NUMERIC", mode="REQUIRED", description="a random number between 0 and 1. Facilitates repeatable data sampling without the need for a hash key ")
    ]
    table_ref = dataset_ref.table(stock_symbol.upper())
    table = bigquery.Table(table_ref, schema=schema)
    
    table.time_partitioning = bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field="quote_date")
   
    table = client.create_table(table)  # API request

    print("Created table {}".format(table.full_table_id))

In [ ]:
#create database
query = """

"""

In [14]:
def create_option_bq_schema(underlying_symbol):
    """
    Create a BigQuery table in the stock_options dataset for the supplied underlying symbol
    """
    schema = """
        CREATE TABLE IF NOT EXISTS expiry-week.option_quotes.{}
        (
            quote_date DATE NOT NULL OPTIONS(description="market closing date for which the data was captured"),  
            expiry_date DATE NOT NULL OPTIONS(description="date on which the options expire"),
            days_to_expiry INT64 NOT NULL OPTIONS(description="calendar days between quote_date and expiry_date not including the quote_date"),
            underlying_price NUMERIC NOT NULL OPTIONS(description="the closing price of the underlying asset"),
            atm_price NUMERIC NOT NULL OPTIONS(description="adjusted underlying price such that the implied volatility of calls an puts are the same"),
            atm_iv NUMERIC NOT NULL OPTIONS(description="the implied volatility for both calls and puts using the atm_price as the current price"),
            strike_prices STRUCT
            <
                strike_price NUMERIC NOT NULL OPTIONS(description="price at which an option can be excercised"),
                call_bid NUMERIC NOT NULL OPTIONS(description="closing bid price for the call option"),
                call_ask NUMERIC NOT NULL OPTIONS(description="closing ask price for the call option"),
                call_volume NUMERIC NOT NULL OPTIONS(description="number of call contracts traded"),
                call_open_interest NUMERIC OPTIONS(description="number of open call contracts"),
                call_moneyness NUMERIC NOT NULL OPTIONS(description="probability of call option closing in the money on expiry date (based on atm_iv)"),
                call_iv NUMERIC NOT NULL OPTIONS(description="implied volatiity of the call option using the midpoint between the call bid and call ask price"),
                put_bid NUMERIC NOT NULL OPTIONS(description="closing bid price for the put option"),
                put_ask NUMERIC NOT NULL OPTIONS(description="closing ask price for the put option"),
                put_volume NUMERIC NOT NULL OPTIONS(description="number of put contracts traded"),
                put_open_interest NUMERIC NOT NULL OPTIONS(description="number of open call contracts"),
                put_moneyness NUMERIC NOT NULL OPTIONS(description="probability of put option closing in the money on expiry date (based on atm_iv)"),
                put_iv NUMERIC NOT NULL OPTIONS(description="implied volatiity of the put option using the midpoint between the put bid and put ask price")
            >,
            sampling_key NUMERIC NOT NULL OPTIONS(description="a random number between 0 and 1. Facilitates repeatable data sampling without the need for a hash key")
        )
        PARTITION BY quote_date
        CLUSTER BY expiry_date
        OPTIONS
            (description="DAILY STOCK OPTION QUOTES")
        """

    return schema.format(underlying_symbol)

In [15]:
print(create_option_bq_schema("SPY"))


        CREATE TABLE IF NOT EXISTS expiry-week.option_quotes.SPY
        (
            quote_date DATE NOT NULL OPTIONS(description="market closing date for which the data was captured"),  
            expiry_date DATE NOT NULL OPTIONS(description="date on which the options expire"),
            days_to_expiry INT64 NOT NULL OPTIONS(description="calendar days between quote_date and expiry_date not including the quote_date"),
            underlying_price NUMERIC NOT NULL OPTIONS(description="the closing price of the underlying asset"),
            atm_price NUMERIC NOT NULL OPTIONS(description="adjusted underlying price such that the implied volatility of calls an puts are the same"),
            atm_iv NUMERIC NOT NULL OPTIONS(description="the implied volatility for both calls and puts using the atm_price as the current price"),
            strike_prices STRUCT
            <
                strike_price NUMERIC NOT NULL OPTIONS(description="price at which an option can be excercised")

In [16]:
%bq --version

UsageError: Line magic function `%bq` not found.
